In [ ]:
from flask import Flask, render_template
import pandas as pd
from datetime import datetime, timedelta
from prettytable import PrettyTable
import random

app = Flask(__name__)

airport = 'MYR'

def update_table(load_airport):

    # Read data from CSV files
    data = pd.read_csv("2008.csv")
    carrier = pd.read_csv("AirlineCodes.csv")
    airport = pd.read_csv("airports.csv")

    # Get current date and time
    current_month = datetime.now().month
    current_day = datetime.now().day
    current_time = datetime.now().time()

    # Filter itinerary for current day, get needed columns
    itinerary = data.loc[(data['Month'] == current_month) & (data['DayofMonth'] == current_day), ['UniqueCarrier', 'DepTime', 'ArrTime', 'FlightNum', 'Origin', 'Dest']]
    itinerary = itinerary.dropna()

    # Create mapping dictionaries for carrier codes and airport IDs
    carrier_mapping = dict(zip(carrier['Code'], carrier['Name']))
    airport_mapping = dict(zip(airport['ident'].str.lstrip('K'), airport['municipality'].str.split('/ ').str[0]))

    # Extract delay and cancellation data
    delays = data[['DepDelay']]
    cancellations = data[['Cancelled']]

    # Make a gate number based on FlightNum
    def get_gate(FlightNum):
        FlightNum = str(FlightNum)  # Convert to string
        first_digit = int(FlightNum[0])
        last_two_digits = int(FlightNum[-2:])

        letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
        gate_letter = letters[first_digit - 1]

        gate_number = first_digit + last_two_digits

        if gate_number > 99:
            gate_number = int(gate_number / 1.5)

        return f"{gate_letter}{gate_number}"

    # Parse time strings into datetime.time objects
    def parse_time(time_str):
        try:
            return datetime.strptime(str(time_str).split('.')[0], "%H%M").time()
        except ValueError:
            pass

    # The angry customer function
    def has_delay(row): 
        for column in delays:
            if pd.notna(data.loc[row.name, column]) and data.loc[row.name, column] >= 15:
                return True
        return False

    # The super angry customer function
    def cancelled(row):
        for column in cancellations:
            if pd.notna(data.loc[row.name, column]) and data.loc[row.name, column] != 0:
                return True
        return False

    # Better speed walk through the airport, we're boarding
    def boarding(row):
        dep_time = parse_time(row['DepTime'])
        if dep_time is not None:
            boarding_time = datetime.combine(datetime.today(), dep_time) - timedelta(minutes=45)
            return boarding_time.time() <= current_time <= dep_time
        return False

    # No ma'am, we will not come back to the gate to get you
    def departed(row):
        dep_time = parse_time(row['DepTime'])
        if dep_time is not None:
            push_back = datetime.combine(datetime.today(), dep_time) - timedelta(minutes=10)
            return push_back.time() <= current_time 
        return False
    
    # Are they here yet?
    def landed(row):
        arr_time = parse_time(row['ArrTime'])
        dep_time = parse_time(row['DepTime'])
        if arr_time is not None and current_time >= dep_time and current_time >= arr_time:
            return True
        return False
            
    # Get airline name based on the carrier code
    def get_airline(carrier_code):
        if carrier_code in carrier_mapping:
            return carrier_mapping[carrier_code]
        return "Void"

    # Get airport name based on the airport ID
    def get_airport(id):
        if id in airport_mapping:
            return airport_mapping[id]
        return "Void"

    # Get status of flight
    def get_status(row, airport):
        if has_delay(row) and not boarding(row) and not departed(row) and not landed(row):
            return "Delayed"
        elif cancelled(row):
            return "Cancelled"
        elif boarding(row) and not departed(row):
            return "Boarding"
        elif departed(row) and row['Dest'] != load_airport:
            return "Departed"
        elif landed(row):
            return "Landed"
        else:
            return "On Time"

    # Format time objects as strings
    def format_time(time):
        return time.strftime("%I:%M %p")

    # Generate the arrival board
    def generate_arrival_board(rows):
        arrivals = []
        for _, row in rows.iterrows():
            arr_time = parse_time(row['ArrTime'])
            dep_time = parse_time(row['DepTime'])
            origin = get_airport(row['Origin'])
            if arr_time is not None and dep_time is not None and origin != 'Void':
                arr_display = datetime.combine(datetime.today(), arr_time) + timedelta(minutes=60)
                if current_time >= dep_time and current_time <= arr_display.time():
                    arrivals.append({
                        'Airline': get_airline(row['UniqueCarrier']),
                        'Flight': row['FlightNum'],
                        'Origin': get_airport(row['Origin']),
                        'Status': get_status(row, load_airport),
                        'Time': format_time(arr_time),
                        'Gate': get_gate(row['FlightNum'])
                    })
        return arrivals

    # Generate the departure board as a list of dictionaries
    def generate_departure_board(rows):
        departures = []
        for _, row in rows.iterrows():
            dep_time = parse_time(row['DepTime'])
            destination = get_airport(row['Dest'])
            if dep_time is not None and destination != 'Void':
                dep_display = datetime.combine(datetime.today(), dep_time) + timedelta(minutes=60)
                if current_time <= dep_display.time():
                    departures.append({
                        'Airline': get_airline(row['UniqueCarrier']),
                        'Flight': row['FlightNum'],
                        'Destination': get_airport(row['Dest']),
                        'Status': get_status(row, load_airport),
                        'Time': format_time(dep_time),
                        'Gate': get_gate(row['FlightNum'])
                    })
        return departures
    

    # Create variables
    arrival_board = generate_arrival_board(itinerary[itinerary['Dest'].eq(load_airport)])
    departure_board = generate_departure_board(itinerary[itinerary['Origin'].eq(load_airport)])
    display_time = format_time(current_time)

    return arrival_board, departure_board

    #print(display_time)

    #print("\nArrivals:")
    #print(arrival_board)

    #print("\nDepartures:")
    #print(departure_board)


@app.route('/')
def index():
    arrival_board, departure_board = update_table(airport)
    current_time = datetime.now().strftime("%I:%M %p")
    airport_code = airport
    
    return render_template('index.html', arrivals=arrival_board, departures=departure_board, current_time=current_time, airport_code=airport_code)


if __name__ == '__main__':
    app.run()

 
 
 
 
 
 

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Jul/2023 20:01:20] "GET / HTTP/1.1" 200 -
